In [6]:
import os
import numpy as np
import librosa
import soundfile as sf
from tqdm import tqdm

class AudioPreprocessor:
    def __init__(self, input_folder, output_folder, segment_duration=60):
        """
        Inicializa el preprocesador de audio
        
        :param input_folder: Carpeta con los archivos WAV originales
        :param output_folder: Carpeta donde se guardarán los archivos procesados
        :param segment_duration: Duración de cada segmento en segundos (default: 60)
        """
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.segment_duration = segment_duration
        
        # Crear carpeta de salida si no existe
        os.makedirs(output_folder, exist_ok=True)

    def normalize_audio(self, audio):
        """Normaliza el audio a un rango de -1 a 1"""
        return librosa.util.normalize(audio)

    def segment_audio(self, audio, sr):
        """
        Divide el audio en segmentos de duración específica
        
        :param audio: Array de numpy con el audio
        :param sr: Frecuencia de muestreo
        :return: Lista de segmentos de audio
        """
        # Calcular el número de muestras por segmento
        segment_length = int(sr * self.segment_duration)
        
        # Calcular número de segmentos completos
        num_segments = len(audio) // segment_length
        
        segments = []
        for i in range(num_segments):
            start = i * segment_length
            end = start + segment_length
            segment = audio[start:end]
            segments.append(segment)
        
        # Manejar el último segmento si es necesario
        remaining = audio[num_segments * segment_length:]
        if len(remaining) > sr * 5:  # Solo incluir si es mayor a 5 segundos
            segments.append(remaining)
        
        return segments

    def process_file(self, filename):
        """
        Procesa un archivo de audio individual
        
        :param filename: Nombre del archivo a procesar
        """
        try:
            # Cargar el archivo
            file_path = os.path.join(self.input_folder, filename)
            print(f"\nProcesando: {filename}")
            
            # Cargar audio
            audio, sr = librosa.load(file_path, sr=None)
            print(f"Duración original: {len(audio)/sr:.2f} segundos")
            
            # Normalizar
            audio_normalized = self.normalize_audio(audio)
            
            # Segmentar
            segments = self.segment_audio(audio_normalized, sr)
            print(f"Número de segmentos: {len(segments)}")
            
            # Guardar segmentos
            base_name = os.path.splitext(filename)[0]
            for i, segment in enumerate(segments, 1):
                # Crear nombre de archivo
                new_filename = f"{base_name}_part{i:03d}.wav"
                output_path = os.path.join(self.output_folder, new_filename)
                
                # Guardar archivo
                sf.write(output_path, segment, sr, subtype='PCM_16')
                
            return len(segments)
            
        except Exception as e:
            print(f"Error procesando {filename}: {str(e)}")
            return 0

    def process_folder(self):
        """Procesa todos los archivos WAV en la carpeta de entrada"""
        try:
            # Obtener lista de archivos WAV
            wav_files = [f for f in os.listdir(self.input_folder) 
                        if f.lower().endswith('.wav')]
            
            if not wav_files:
                print("No se encontraron archivos WAV en la carpeta de entrada.")
                return
            
            print(f"Encontrados {len(wav_files)} archivos WAV")
            
            # Procesar cada archivo
            total_segments = 0
            for filename in tqdm(wav_files, desc="Procesando archivos"):
                segments = self.process_file(filename)
                total_segments += segments
            
            print(f"\nProcesamiento completado:")
            print(f"- Archivos procesados: {len(wav_files)}")
            print(f"- Segmentos totales generados: {total_segments}")
            
        except Exception as e:
            print(f"Error durante el procesamiento: {str(e)}")

# Uso del script
if __name__ == "__main__":
    input_folder = "/Volumes/Nexus/GRABACIONES DE CAMPO/2024/OCTUBRE/castrelos dom-27-oct-24/CRUDO/"
    output_folder = "/Volumes/Nexus/GRABACIONES DE CAMPO/2024/OCTUBRE/castrelos dom-27-oct-24/EDICIONES/"
    
    # Crear instancia del preprocesador
    processor = AudioPreprocessor(
        input_folder=input_folder,
        output_folder=output_folder,
        segment_duration=60  # Duración en segundos de cada segmento
    )
    
    # Procesar archivos
    processor.process_folder()

Encontrados 6 archivos WAV


Procesando archivos:   0%|          | 0/6 [00:00<?, ?it/s]


Procesando: 6.WAV
Duración original: 298.26 segundos
Número de segmentos: 5


Procesando archivos:  17%|█▋        | 1/6 [00:03<00:15,  3.18s/it]


Procesando: 5.WAV
Duración original: 389.42 segundos
Número de segmentos: 7


...


Procesando: 3.WAV
Duración original: 368.50 segundos
Número de segmentos: 7


Procesando archivos:  83%|████████▎ | 5/6 [00:19<00:04,  4.08s/it]


Procesando: 2.WAV
Duración original: 475.16 segundos
Número de segmentos: 8


Procesando archivos: 100%|██████████| 6/6 [00:24<00:00,  4.16s/it]


Procesamiento completado:
- Archivos procesados: 6
- Segmentos totales generados: 41
